In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from ipywidgets import widgets

In [3]:
%load_ext line_profiler

In [4]:
from robofan_main_v2 import RoboFan

In [5]:
import ipywidgets as widgets

file = open('test.jpg', "rb")
image = file.read()

result_widget = widgets.Image(
    value=image,
    format='png',
    width=1024,
    height=768,
)


text_widget = widgets.HTML(
    value='Init',
    placeholder='',
    description='',
)

out = widgets.VBox([text_widget, result_widget])
display(out)

In [ ]:
robofan = RoboFan()

try:
    camera.close()
except:
    pass
        
# import the necessary packages
from picamera.array import PiRGBArray
from picamera import PiCamera
import time
import cv2
# initialize the camera and grab a reference to the raw camera capture
camera = PiCamera()
# camera.resolution = (640,480)
# rawCapture = PiRGBArray(camera, size=(640, 480))
rawCapture = PiRGBArray(camera)
# allow the camera to warmup
time.sleep(0.1)
# grab an image from the camera

while True:
    camera.capture(rawCapture, format="bgr")
    image = rawCapture.array

#     result_widget.value = cv2.imencode('.jpg', image)[1].tostring()
    rawCapture.truncate(0)
    
    people = robofan.process_image(image, result_widget=result_widget)
#     print(people)

Person detected at 87: moving right
0 1  0.29 seconds
Person detected at 87: moving right
0 1  0.23 seconds
Person detected at 84: moving right
0 1  0.23 seconds
Person detected at 85: moving right
0 1  0.28 seconds
Person detected at 83: moving right
0 1  0.22 seconds
Person detected at 85: moving right
0 1  0.24 seconds
Person detected at 87: moving right
0 1  0.23 seconds
Person detected at 86: moving right
0 1  0.26 seconds
Person detected at 87: moving right
0 1  0.22 seconds
Person detected at 86: moving right
0 1  0.27 seconds
Person detected at 83: moving right
0 1  0.22 seconds
Person detected at 84: moving right
0 1  0.22 seconds
Person detected at 84: moving right
0 1  0.25 seconds
Person detected at 87: moving right
0 1  0.22 seconds
Person detected at 90: moving right
0 2  0.22 seconds
Person detected at 86: moving right
0 1  0.28 seconds
Person detected at 84: moving right
0 1  0.26 seconds
Person detected at 84: moving right
0 1  0.22 seconds
Person detected at 84: movin

In [6]:
## Setup model
import os
import argparse
import cv2
import numpy as np
import sys
import time
from threading import Thread
import importlib.util
import matplotlib.pyplot as plt

MODEL_NAME = 'Sample_TFLite_model'
GRAPH_NAME = 'detect.tflite'
LABELMAP_NAME = 'labelmap.txt'
min_conf_threshold = float(0.5)
# resW, resH = args.resolution.split('x')
imW, imH = int(1280), int(720)
use_TPU = False

# Import TensorFlow libraries
# If tflite_runtime is installed, import interpreter from tflite_runtime, else import from regular tensorflow
# If using Coral Edge TPU, import the load_delegate library
pkg = importlib.util.find_spec('tflite_runtime')
if pkg:
    from tflite_runtime.interpreter import Interpreter
    if use_TPU:
        from tflite_runtime.interpreter import load_delegate
else:
    from tensorflow.lite.python.interpreter import Interpreter
    if use_TPU:
        from tensorflow.lite.python.interpreter import load_delegate

# If using Edge TPU, assign filename for Edge TPU model
if use_TPU:
    # If user has specified the name of the .tflite file, use that name, otherwise use default 'edgetpu.tflite'
    if (GRAPH_NAME == 'detect.tflite'):
        GRAPH_NAME = 'edgetpu.tflite'       

# Get path to current working directory
CWD_PATH = os.getcwd()

# Path to .tflite file, which contains the model that is used for object detection
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,GRAPH_NAME)

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,MODEL_NAME,LABELMAP_NAME)

# Load the label map
with open(PATH_TO_LABELS, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

# Have to do a weird fix for label map if using the COCO "starter model" from
# https://www.tensorflow.org/lite/models/object_detection/overview
# First label is '???', which has to be removed.
if labels[0] == '???':
    del(labels[0])

# Load the Tensorflow Lite model.
# If using Edge TPU, use special load_delegate argument
if use_TPU:
    interpreter = Interpreter(model_path=PATH_TO_CKPT,
                              experimental_delegates=[load_delegate('libedgetpu.so.1.0')])
    print(PATH_TO_CKPT)
else:
    interpreter = Interpreter(model_path=PATH_TO_CKPT)

interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

floating_model = (input_details[0]['dtype'] == np.float32)

input_mean = 127.5
input_std = 127.5

# Initialize frame rate calculation
frame_rate_calc = 1
freq = cv2.getTickFrequency()

# Initialize video stream
# videostream = VideoStream(resolution=(imW,imH),framerate=30).start()
# time.sleep(1)


In [9]:
# Load image and resize to expected shape [1xHxWx3]
import glob
images = glob.glob('test_captures/*')

for impath in images:
    image = cv2.imread(impath)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    imH, imW, _ = image.shape 
    image_resized = cv2.resize(image_rgb, (width, height))
    input_data = np.expand_dims(image_resized, axis=0)

    # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
    if floating_model:
        input_data = (np.float32(input_data) - input_mean) / input_std

    # Perform the actual detection by running the model with the image as input
    interpreter.set_tensor(input_details[0]['index'],input_data)
    interpreter.invoke()

    # Retrieve detection results
    boxes = interpreter.get_tensor(output_details[0]['index'])[0] # Bounding box coordinates of detected objects
    classes = interpreter.get_tensor(output_details[1]['index'])[0] # Class index of detected objects
    scores = interpreter.get_tensor(output_details[2]['index'])[0] # Confidence of detected objects
    #num = interpreter.get_tensor(output_details[3]['index'])[0]  # Total number of detected objects (inaccurate and not needed)
    boxes, classes, scores

    # Loop over all detections and draw detection box if confidence is above minimum threshold
    for i in range(len(scores)):
        if ((scores[i] > min_conf_threshold) and (scores[i] <= 1.0)):

            
            object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
            if object_name == 'person':

                # Get bounding box coordinates and draw box
                # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
                ymin = int(max(1,(boxes[i][0] * imH)))
                xmin = int(max(1,(boxes[i][1] * imW)))
                ymax = int(min(imH,(boxes[i][2] * imH)))
                xmax = int(min(imW,(boxes[i][3] * imW)))

                cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)
                
                # Draw label
                label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
                labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
                label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
                cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
                cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text



    result_widget.value = cv2.imencode('.jpg', image)[1].tostring()

In [ ]:
robofan.run(result_widget=result_widget, text_widget=text_widget)

In [28]:
from stepper.driver import move

move(0.2, 1, rev_per_sec=0.5)
robofan.camera.capture('test.jpg')

file = open('test.jpg', "rb")
image = file.read()
# result_widget.value = image
image

b'\xff\xd8\xff\xe1d Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\n\x01\x00\x00\x04\x00\x00\x00\x01\x00\x00\x02\x80\x01\x01\x00\x04\x00\x00\x00\x01\x00\x00\x01\xe0\x01\x0f\x00\x02\x00\x00\x00\x0c\x00\x00\x00\x86\x01\x10\x00\x02\x00\x00\x00\n\x00\x00\x00\x92\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00\x9c\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\xa4\x01(\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x012\x00\x02\x00\x00\x00\x14\x00\x00\x00\xac\x02\x13\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\xc0\x00\x00\x03\xa2RaspberryPi\x00RP_imx219\x00\x00\x00\x00H\x00\x00\x00\x01\x00\x00\x00H\x00\x00\x00\x012020:12:20 12:24:33\x00\x00\x17\x82\x9a\x00\x05\x00\x00\x00\x01\x00\x00\x01\xda\x82\x9d\x00\x05\x00\x00\x00\x01\x00\x00\x01\xe2\x88"\x00\x03\x00\x00\x00\x01\x00\x03\x00\x00\x88\'\x00\x03\x00\x00\x00\x01\x01@\x00\x00\x90\x00\x00\x07\x00\x00\x00\x040220\x90\x03\x00\x02\x00\x00\x00\x14\x00\x00\x01\xea\x90\x04\x00\x02\x00\x00\x00\x14\x00\x00\x01\xfe\x91\x01\x00\x07\x00\

In [7]:
while True:
    robofan.camera.capture('test.jpg')

    file = open('test.jpg', "rb")
    image = file.read()
    result_widget.value = image

KeyboardInterrupt: 

In [8]:
robofan.camera.capture('test.jpg')

file = open('test.jpg', "rb")
image = file.read()
result_widget.value = image

In [ ]:
# test_run(result_widget=result_widget, text_widget=text_widget)
robofan.test_run(result_widget=None, text_widget=text_widget)

In [ ]:
%lprun -f robofan.detect_people robofan.test_run(text_widget=text_widget)